In [19]:
import tensorrt as trt
from tensorrt_model import TRTModel
from ssd_tensorrt import load_plugins, parse_boxes,TRT_INPUT_NAME, TRT_OUTPUT_NAME
import ctypes
import numpy as np
import cv2
import os
import ctypes


mean = 255.0 * np.array([0.5, 0.5, 0.5])
stdev = 255.0 * np.array([0.5, 0.5, 0.5])


def bgr8_to_ssd_input(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1)).astype(np.float32)
    x -= mean[:, None, None]
    x /= stdev[:, None, None]
    return x[None, ...]

class ObjectDetector(object):
    
    def __init__(self, engine_path, preprocess_fn=bgr8_to_ssd_input):
        logger = trt.Logger()
        trt.init_libnvinfer_plugins(logger, '')
        load_plugins()
        self.trt_model = TRTModel(engine_path, input_names=[TRT_INPUT_NAME],output_names=[TRT_OUTPUT_NAME, TRT_OUTPUT_NAME + '_1'])
        self.preprocess_fn = preprocess_fn
        
    def execute(self, *inputs):
        trt_outputs = self.trt_model(self.preprocess_fn(*inputs))
        return parse_boxes(trt_outputs)
    
    def __call__(self, *inputs):
        return self.execute(*inputs)


In [2]:
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [3]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of these two values will be captured by traitlets
    color_value = traitlets.Any()
    depth_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolubg_removedtion for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        self.stop_flag = False
        
        #Create an align object
        self.align = rs.align(rs.stream.color)
        
        #start capture 
        profile = self.pipeline.start(self.configuration)
        
        # clipping distance 
        depth_sensor = profile.get_device().first_depth_sensor()
        depth_scale = depth_sensor.get_depth_scale()
                        
        #the first depth and color image
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()
        
        # align the frames
        aligned_frames = self.align.process(frames)

        color_frame = aligned_frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.color_value = image

        depth_frame = aligned_frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_image    

    def _capture_frames(self):
        while(self.stop_flag==False):
            frames = self.pipeline.wait_for_frames()
            
            # align the frames
            aligned_frames = self.align.process(frames)
            
            color_frame = aligned_frames.get_color_frame()
            image = np.asanyarray(color_frame.get_data())
            

            depth_frame = aligned_frames.get_depth_frame()              
            depth_image = np.asanyarray(depth_frame.get_data())            
            
            self.color_value = image
            
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap
            self.depth_image = depth_image
            
            
    def start(self):
        if not hasattr(self, 'thread') or not self.thread.isAlive():
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        self.stop_flag = True
        if hasattr(self, 'thread'):
            self.thread.join()        
        if self.pipeline_started:
            self.pipeline.stop()

def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [25]:
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
from mochris import RangeSensors

range_sensors = RangeSensors(front=True, front_range=2, front_roi=3, fiat=0)

width = 640
height = 480

display_depth = widgets.Image(format='jpeg') #, height=480)
image_widget = widgets.Image(format='jpeg') #, height=300)

label_widget = widgets.IntText(value=1, description='tracked label')

distance_slider = widgets.IntSlider(description='Obstacle', min=40, max=4000, orientation='horizontal')

display(widgets.VBox([
    widgets.HBox([image_widget, display_depth]),
    label_widget,
    distance_slider
]))

import time
from RobotClass import Robot

#initialize the Robot class
robot = Robot()
bbox = None

MIN_LEFT = 270
MIN_RIGHT = 370
TURN_SPEED = 0.4

FORWARD_SPEED = 0.4
FOLOW_DEPTH = 900

def processing(change):
    image = change['new']
    imgsized= cv2.resize(image,(300,300))
    # compute all detected objects
    detections = model(imgsized)
    
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    
    for det in matching_detections:
        bbox = det['bbox']
            
        start_point = (int(width * bbox[0]), int(height * bbox[1]))
        end_point = (int(width * bbox[2]), int(height * bbox[3]))
        
        center = (int((start_point[0] + end_point[0])/2), int((start_point[1] + end_point[1])/2))
        
        cv2.rectangle(image, start_point, end_point, (255, 0, 0), 2)
       
    if len(matching_detections)==0:
        robot.stop()
    elif center[0] >= 0 and center[0] < width and center[1] >= 0 and center[1] < height:
        global distance_slider
        depth_box = camera.depth_image[start_point[1]:end_point[1], start_point[0]:end_point[0]]
        
        depth_box[depth_box<100] = 0
        depth_box[depth_box>5000] = 0
        
        depth_box = depth_box[depth_box!=0]
        
        depth = 0
        if len(depth_box) != 0:
            depth = depth_box.min()
            distance_slider.value = depth
        
        cv2.circle(image, center, 5, (0,0,255), 2)
        
        image[center[1],center[0]] = (0,0,255)
        
        if center[0] < MIN_LEFT:
            robot.left(TURN_SPEED)
        elif center[0] > MIN_RIGHT:
            robot.right(TURN_SPEED)
        elif depth != 0 and depth > FOLOW_DEPTH:
            robot.forward(FORWARD_SPEED)
        else:
            robot.stop()
        
    
    image_widget.value = bgr8_to_jpeg(image)
    
#the camera.observe function will monitor the color_value variable. If this value changes, the excecute function will be excuted.
processing({'new': camera.color_value})
camera.observe(processing, names='color_value')


def bgr8_to_jpeg(value,quality=75):
    return bytes(cv2.imencode('.jpg',value)[1])


#link the camera.depth_value to the display_depth
camera_link_depth = traitlets.dlink((camera, 'depth_value'), (display_depth, 'value'), transform=bgr8_to_jpeg)



In [26]:
camera.unobserve_all()
time.sleep(1.0)
robot.stop()

In [ ]:
  
            #we only consider the central area of the vision sensor
            depth_image[:190,:]=0
            depth_image[290:,:]=0
            depth_image[:,:160]=0
            depth_image[:,480:]=0
            
            #For object avoidance, we don't consider the distance that are lower than 100mm or bigger than 1000mm
            depth_image[depth_image<100]=0
            depth_image[depth_image>1000]=0
            
            #If all of the values in the depth image is 0, the depth[depth!=0] command will fail
            #we set a specific value here to prevent this failure
            depth_image[0,0]=2000

#             ss=np.zeros((480,640),dtype=np.uint8)    
#             ss[depth_image!=0]=255
#             cv2.imshow('ss',ss)
            
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            if (depth_image[depth_image!=0].min()<500):
                cv2.putText(depth_colormap, 'warning!!!', (320,240), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                self.warning_flag=1
            else:
                self.warning_flag=0

In [ ]:
del camera